# Quickstart: Pandas API on Spark

This is a short introduction to pandas API on Spark, geared mainly for new users. This notebook shows you some key differences between pandas and pandas API on Spark. You can run this examples by yourself in 'Live Notebook: pandas API on Spark' at [the quickstart page](https://spark.apache.org/docs/latest/api/python/getting_started/index.html).

Customarily, we import pandas API on Spark as follows:

In [1]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

## Object Creation



Creating a pandas-on-Spark Series by passing a list of values, letting pandas API on Spark create a default integer index:

In [2]:
s = ps.Series([1, 3, 5, np.nan, 6, 8])

In [4]:
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

Creating a pandas-on-Spark DataFrame by passing a dict of objects that can be converted to series-like.

In [4]:
psdf = ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [5]:
psdf

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


Creating a pandas DataFrame by passing a numpy array, with a datetime index and labeled columns:

In [6]:
dates = pd.date_range('20130101', periods=6)

In [7]:
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [8]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

In [9]:
pdf

,A,B,C,D
2013-01-01,0.841545,-1.984537,-1.204401,0.825285
2013-01-02,1.223045,0.544392,-0.066700,-0.987072
2013-01-03,1.771369,0.262683,0.137983,-1.057627
2013-01-04,-0.338881,2.245503,1.538394,-1.472213
2013-01-05,1.134308,-0.404942,-0.566942,0.805066
2013-01-06,0.003309,0.417175,1.140308,-0.082665


Now, this pandas DataFrame can be converted to a pandas-on-Spark DataFrame

In [10]:
psdf = ps.from_pandas(pdf)

In [11]:
type(psdf)

pyspark.pandas.frame.DataFrame

It looks and behaves the same as a pandas DataFrame.

In [12]:
psdf

,A,B,C,D
2013-01-01,0.841545,-1.984537,-1.204401,0.825285
2013-01-02,1.223045,0.544392,-0.066700,-0.987072
2013-01-03,1.771369,0.262683,0.137983,-1.057627
2013-01-04,-0.338881,2.245503,1.538394,-1.472213
2013-01-05,1.134308,-0.404942,-0.566942,0.805066
2013-01-06,0.003309,0.417175,1.140308,-0.082665


Also, it is possible to create a pandas-on-Spark DataFrame from Spark DataFrame easily.

Creating a Spark DataFrame from pandas DataFrame

In [13]:
spark = SparkSession.builder.getOrCreate()

In [14]:
sdf = spark.createDataFrame(pdf)

In [15]:
sdf.show()

+--------------------+--------------------+-------------------+--------------------+
|                   A|                   B|                  C|                   D|
+--------------------+--------------------+-------------------+--------------------+
|  0.8415449122302842| -1.9845368583477034|-1.2044014179899183|  0.8252847032795035|
|  1.2230450446691083|  0.5443915666052449|-0.0667004649993457|  -0.987071579707262|
|  1.7713690797755324|  0.2626833041943157|0.13798324666982142| -1.0576271409472096|
| -0.3388809007122013|   2.245502623261706| 1.5383937559692853|  -1.472212899094161|
|  1.1343076150972582|-0.40494193696849984|-0.5669415775623219|  0.8050655219920475|
|0.003309074354972...|  0.4171754962012393| 1.1403083932097862|-0.08266503331705147|
+--------------------+--------------------+-------------------+--------------------+



Creating pandas-on-Spark DataFrame from Spark DataFrame.

In [16]:
psdf = sdf.to_pandas_on_spark()

In [17]:
psdf

,A,B,C,D
0,0.841545,-1.984537,-1.204401,0.825285
1,1.223045,0.544392,-0.066700,-0.987072
2,1.771369,0.262683,0.137983,-1.057627
3,-0.338881,2.245503,1.538394,-1.472213
4,1.134308,-0.404942,-0.566942,0.805066
5,0.003309,0.417175,1.140308,-0.082665


Having specific [dtypes](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-dtypes) . Types that are common to both Spark and pandas are currently supported.

In [18]:
psdf.dtypes

A    float64
B    float64
C    float64
D    float64
dtype: object

Here is how to show top rows from the frame below.

Note that the data in a Spark dataframe does not preserve the natural order by default. The natural order can be preserved by setting `compute.ordered_head` option but it causes a performance overhead with sorting internally.

In [19]:
psdf.head()

,A,B,C,D
0,0.841545,-1.984537,-1.204401,0.825285
1,1.223045,0.544392,-0.066700,-0.987072
2,1.771369,0.262683,0.137983,-1.057627
3,-0.338881,2.245503,1.538394,-1.472213
4,1.134308,-0.404942,-0.566942,0.805066


Displaying the index, columns, and the underlying numpy data.

In [20]:
psdf.index

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [21]:
psdf.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [22]:
psdf.to_numpy()

array([[ 0.84154491, -1.98453686, -1.20440142,  0.8252847 ],
       [ 1.22304504,  0.54439157, -0.06670046, -0.98707158],
       [ 1.77136908,  0.2626833 ,  0.13798325, -1.05762714],
       [-0.3388809 ,  2.24550262,  1.53839376, -1.4722129 ],
       [ 1.13430762, -0.40494194, -0.56694158,  0.80506552],
       [ 0.00330907,  0.4171755 ,  1.14030839, -0.08266503]])

Showing a quick statistic summary of your data

In [23]:
psdf.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,0.772449,0.180046,0.163107,-0.328204
std,0.795390,1.377646,1.029838,0.994992
min,-0.338881,-1.984537,-1.204401,-1.472213
25%,0.003309,-0.404942,-0.566942,-1.057627
50%,0.841545,0.262683,-0.066700,-0.987072
75%,1.223045,0.544392,1.140308,0.805066
max,1.771369,2.245503,1.538394,0.825285


Transposing your data

In [24]:
psdf.T

,0,1,2,3,4,5
A,0.841545,1.223045,1.771369,-0.338881,1.134308,0.003309
B,-1.984537,0.544392,0.262683,2.245503,-0.404942,0.417175
C,-1.204401,-0.066700,0.137983,1.538394,-0.566942,1.140308
D,0.825285,-0.987072,-1.057627,-1.472213,0.805066,-0.082665


Sorting by its index

In [25]:
psdf.sort_index(ascending=False)

,A,B,C,D
5,0.003309,0.417175,1.140308,-0.082665
4,1.134308,-0.404942,-0.566942,0.805066
3,-0.338881,2.245503,1.538394,-1.472213
2,1.771369,0.262683,0.137983,-1.057627
1,1.223045,0.544392,-0.066700,-0.987072
0,0.841545,-1.984537,-1.204401,0.825285


Sorting by value

In [26]:
psdf.sort_values(by='B')

,A,B,C,D
0,0.841545,-1.984537,-1.204401,0.825285
4,1.134308,-0.404942,-0.566942,0.805066
2,1.771369,0.262683,0.137983,-1.057627
5,0.003309,0.417175,1.140308,-0.082665
1,1.223045,0.544392,-0.066700,-0.987072
3,-0.338881,2.245503,1.538394,-1.472213


## Missing Data
Pandas API on Spark primarily uses the value `np.nan` to represent missing data. It is by default not included in computations. 


In [27]:
pdf1 = pdf.reindex(index=dates[0:4], columns=list(pdf.columns) + ['E'])

In [28]:
pdf1.loc[dates[0]:dates[1], 'E'] = 1

In [29]:
psdf1 = ps.from_pandas(pdf1)

In [30]:
psdf1

,A,B,C,D,E
2013-01-01,0.841545,-1.984537,-1.204401,0.825285,1.0
2013-01-02,1.223045,0.544392,-0.066700,-0.987072,1.0
2013-01-03,1.771369,0.262683,0.137983,-1.057627,NaN
2013-01-04,-0.338881,2.245503,1.538394,-1.472213,NaN


To drop any rows that have missing data.

In [31]:
psdf1.dropna(how='any')

,A,B,C,D,E
2013-01-01,0.841545,-1.984537,-1.204401,0.825285,1.0
2013-01-02,1.223045,0.544392,-0.066700,-0.987072,1.0


Filling missing data.

In [32]:
psdf1.fillna(value=5)

,A,B,C,D,E
2013-01-01,0.841545,-1.984537,-1.204401,0.825285,1.0
2013-01-02,1.223045,0.544392,-0.066700,-0.987072,1.0
2013-01-03,1.771369,0.262683,0.137983,-1.057627,5.0
2013-01-04,-0.338881,2.245503,1.538394,-1.472213,5.0


## Operations

### Stats
Performing a descriptive statistic:

In [33]:
psdf.mean()

A    0.772449
B    0.180046
C    0.163107
D   -0.328204
dtype: float64

### Spark Configurations

Various configurations in PySpark could be applied internally in pandas API on Spark.
For example, you can enable Arrow optimization to hugely speed up internal pandas conversion. See also <a href="https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html">PySpark Usage Guide for Pandas with Apache Arrow</a> in PySpark documentation.

In [ ]:
prev = spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")  # Keep its default value.
ps.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
%timeit ps.range(300000).to_pandas()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", False)
%timeit ps.range(300000).to_pandas()

In [ ]:
ps.reset_option("compute.default_index_type")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", prev)  # Set its default value back.

## Grouping
By “group by” we are referring to a process involving one or more of the following steps:

- Splitting the data into groups based on some criteria
- Applying a function to each group independently
- Combining the results into a data structure

In [ ]:
psdf = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})

In [ ]:
psdf

Grouping and then applying the [sum()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.groupby.GroupBy.sum.html) function to the resulting groups.

In [ ]:
psdf.groupby('A').sum()

Grouping by multiple columns forms a hierarchical index, and again we can apply the sum function.

In [ ]:
psdf.groupby(['A', 'B']).sum()

## Plotting

In [ ]:
pser = pd.Series(np.random.randn(1000),
                 index=pd.date_range('1/1/2000', periods=1000))

In [ ]:
psser = ps.Series(pser)

In [ ]:
psser = psser.cummax()

In [ ]:
psser.plot()

On a DataFrame, the [plot()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.plot.html) method is a convenience to plot all of the columns with labels:

In [ ]:
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
                   columns=['A', 'B', 'C', 'D'])

In [ ]:
psdf = ps.from_pandas(pdf)

In [ ]:
psdf = psdf.cummax()

In [ ]:
psdf.plot()

For more details, [Plotting](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/frame.html#plotting) documentation.

## Getting data in/out

### CSV

CSV is straightforward and easy to use. See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_csv.html) to write a CSV file and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_csv.html) to read a CSV file.

In [ ]:
psdf.to_csv('foo.csv')
ps.read_csv('foo.csv').head(10)

### Parquet

Parquet is an efficient and compact file format to read and write faster. See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_paruqet.html) to write a Parquet file and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_parquet.html) to read a Parquet file.

In [ ]:
psdf.to_parquet('bar.parquet')
ps.read_parquet('bar.parquet').head(10)

### Spark IO

In addition, pandas API on Spark fully supports Spark's various datasources such as ORC and an external datasource.  See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_orc.html) to write it to the specified datasource and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_orc.html) to read it from the datasource.

In [ ]:
psdf.to_spark_io('zoo.orc', format="orc")
ps.read_spark_io('zoo.orc', format="orc").head(10)

See the [Input/Output](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/io.html) documentation for more details.